In [1]:
import logging

import torch
from torch import nn

import pfrl

import gym
import gym.spaces
import gym.wrappers

In [2]:
import robosuite as suite
from robosuite.wrappers import GymWrapper

In [3]:
### Create the sawyer lift environment
env = GymWrapper(
        suite.make(
            "Lift",
            robots="Sawyer",                # use Sawyer robot
            use_camera_obs=False,           # do not use pixel observations
            has_offscreen_renderer=False,   # not needed since not using pixel obs
            has_renderer=False,              # make sure we can render to the screen
            reward_shaping=True,            # use dense rewards
            control_freq=20,                # control should happen fast enough so that simulation looks smooth
        )
)

adding key: robot0_robot-state
adding key: object-state


C:\Users\91953\Anaconda3\envs\pfrl\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
#ref: https://github.com/ARISE-Initiative/robosuite-benchmark/blob/master/runs/Lift-Sawyer-OSC-POSE-SEED129/Lift_Sawyer_OSC_POSE_SEED129_2020_09_21_20_07_20_0000--s-0/variant.json
### args setup
outdir = "results"
steps = 2 * 10 ** 6 #num_epochs
# num_eval_steps_per_epoch = 2500
eval_interval = 800 # steps/num_eval_steps_per_epoch
eval_n_runs = 1
offset = 3300

render = False
demo = True
load_pretrained = False
pretrained_type = "best"
log_level = logging.INFO
seed = 0
trpo_update_interval = 5000

In [5]:
## initial setup
logging.basicConfig(level=log_level)
pfrl.utils.set_random_seed(seed)
outdir = pfrl.experiments.prepare_output_dir(None, outdir)

In [6]:
def make_env(env, seed, outdir, test=False, monitor=False, render=False):
        # Use different random seeds for train and test envs
        env_seed = 2 ** 32 - 1 - seed if test else seed
        env.seed(env_seed)
        # Cast observations to float32 because our model uses float32
        env = pfrl.wrappers.CastObservationToFloat32(env)
        if monitor:
            env = gym.wrappers.Monitor(env, outdir)
        if render:
            env = pfrl.wrappers.Render(env)
        return env

In [7]:
sawyer_env = make_env(env, seed, outdir)

In [8]:
timestep_limit = 500#sawyer_env.spec.max_episode_steps
obs_space = sawyer_env.observation_space
action_space = sawyer_env.action_space
print("Observation space:", obs_space)
print("Action space:", action_space)

Observation space: Box(-inf, inf, (42,), float32)
Action space: Box(-1.0, 1.0, (8,), float32)


In [9]:
assert isinstance(obs_space, gym.spaces.Box)

In [10]:
# Normalize observations based on their empirical mean and variance
obs_normalizer = pfrl.nn.EmpiricalNormalization(
    obs_space.low.size, clip_threshold=5
)

obs_size = obs_space.low.size
action_size = action_space.low.size
policy = torch.nn.Sequential(
    nn.Linear(obs_size, 64),
    nn.Tanh(),
    nn.Linear(64, 64),
    nn.Tanh(),
    nn.Linear(64, action_size),
    pfrl.policies.GaussianHeadWithStateIndependentCovariance(
        action_size=action_size,
        var_type="diagonal",
        var_func=lambda x: torch.exp(2 * x),  # Parameterize log std
        var_param_init=0,  # log std = 0 => std = 1
    ),
)

C:\Users\91953\Anaconda3\envs\pfrl\lib\site-packages\pfrl\policies\gaussian_policy.py:38: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  torch.tensor(np.broadcast_to(var_param_init, var_size), dtype=torch.float,)


In [11]:
vf = torch.nn.Sequential(
        nn.Linear(obs_size, 64),
        nn.Tanh(),
        nn.Linear(64, 64),
        nn.Tanh(),
        nn.Linear(64, 1),
)

In [12]:
# While the original paper initialized weights by normal distribution,
# we use orthogonal initialization as the latest openai/baselines does.
def ortho_init(layer, gain):
    nn.init.orthogonal_(layer.weight, gain=gain)
    nn.init.zeros_(layer.bias)

ortho_init(policy[0], gain=1)
ortho_init(policy[2], gain=1)
ortho_init(policy[4], gain=1e-2)
ortho_init(vf[0], gain=1)
ortho_init(vf[2], gain=1)
ortho_init(vf[4], gain=1e-2)

In [13]:
# TRPO's policy is optimized via CG and line search, so it doesn't require
# an Optimizer. Only the value function needs it.
vf_opt = torch.optim.Adam(vf.parameters())

In [14]:
 # Hyperparameters in http://arxiv.org/abs/1709.06560
agent = pfrl.agents.TRPO(
    policy=policy,
    vf=vf,
    vf_optimizer=vf_opt,
    obs_normalizer=obs_normalizer,
    gpu=0,
    update_interval=trpo_update_interval,
    max_kl=0.01,
    conjugate_gradient_max_iter=20,
    conjugate_gradient_damping=1e-1,
    gamma=0.995,
    lambd=0.97,
    vf_epochs=5,
    entropy_coef=0,
)

In [ ]:
# Run training experiments
pfrl.experiments.train_agent_with_evaluation(
            agent=agent,
            env=sawyer_env,
            eval_env=make_env(env, seed, outdir),
            outdir=outdir,
            steps=steps,
            step_offset=offset,
            eval_n_steps=None,
            eval_n_episodes=eval_n_runs,
            eval_interval=eval_interval,
            train_max_episode_len=timestep_limit,
    )


INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:3800 episode:0 R:0.1665350169807385
INFO:pfrl.experiments.train_agent:statistics:[('average_value', -0.0005691441), ('average_entropy', 11.351507), ('average_kl', nan), ('average_policy_step_size', nan), ('explained_variance', nan)]
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:4300 episode:1 R:1.3981484230507155
INFO:pfrl.experiments.train_agent:statistics:[('average_value', -0.001488245), ('average_entropy', 11.351506), ('average_kl', nan), ('average_policy_step_size', nan), ('explained_variance', nan)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:0.5202092157166793
INFO:pfrl.experiments.train_agent:The best score is updated -3.4028235e+38 -> 0.5202092157166793
INFO:pfrl.experiments.train_agent:Saved the agent to results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f\best


INFO:pfrl.experiments.train_agent:statistics:[('average_value', 0.021015633), ('average_entropy', 11.329737), ('average_kl', 0.007035945542156696), ('average_policy_step_size', 1.0), ('explained_variance', 0.004829827545341003)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:6.405681833749427
INFO:pfrl.agents.trpo:Line search iteration: 0 step size: 1.0
INFO:pfrl.agents.trpo:Surrogate objective improve: 0.029495913928258233
INFO:pfrl.agents.trpo:KL divergence: 0.0075407009571790695
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:13300 episode:19 R:0.26763957566655155
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 0.04738365), ('average_entropy', 11.329737), ('average_kl', 0.007288323249667883), ('average_policy_step_size', 1.0), ('explained_variance', 0.48417587702128406)]
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:

INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:21800 episode:36 R:3.4203400067937526
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 0.60220987), ('average_entropy', 11.315655), ('average_kl', 0.0069348242444296675), ('average_policy_step_size', 1.0), ('explained_variance', 0.10845825688815303)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:3.3434105901788147
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:22300 episode:37 R:14.72092137877535
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 1.3718466), ('average_entropy', 11.315655), ('average_kl', 0.0069348242444296675), ('average_policy_step_size', 1.0), ('explained_variance', 0.10845825688815303)]
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:22800 episode:38 R:0.20060401139034648
INFO:

INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:30800 episode:54 R:1.5170580714326287
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 0.96609837), ('average_entropy', 11.336156), ('average_kl', 0.00693213315680623), ('average_policy_step_size', 1.0), ('explained_variance', 0.534629162158107)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:3.9412698947578653
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:31300 episode:55 R:20.350485803914324
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 1.1313936), ('average_entropy', 11.336156), ('average_kl', 0.00693213315680623), ('average_policy_step_size', 1.0), ('explained_variance', 0.534629162158107)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:4.118635836844037
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398

INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:27.665168387860867
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:39800 episode:72 R:45.84642543312321
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 4.832034), ('average_entropy', 11.359972), ('average_kl', 0.006923716781394822), ('average_policy_step_size', 1.0), ('explained_variance', 0.6140310255805482)]
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:40300 episode:73 R:27.863730329966938
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 4.710304), ('average_entropy', 11.359972), ('average_kl', 0.006923716781394822), ('average_policy_step_size', 1.0), ('explained_variance', 0.6140310255805482)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:37.221246320475025
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa39

...

INFO:pfrl.experiments.train_agent:statistics:[('average_value', 51.258778), ('average_entropy', 10.945177), ('average_kl', 0.006239177389560561), ('average_policy_step_size', 1.0), ('explained_variance', 0.8523254857203315)]
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:386800 episode:766 R:158.16913827468002
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 50.66243), ('average_entropy', 10.945177), ('average_kl', 0.006239177389560561), ('average_policy_step_size', 1.0), ('explained_variance', 0.8523254857203315)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:145.56943415285076
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:387300 episode:767 R:150.06221618644244
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 55.295773), ('average_entropy', 10.945177), ('average_kl', 0.006239177389560561), ('average_po

INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:148.39730009671342
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:395800 episode:784 R:154.50853433606352
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 50.50222), ('average_entropy', 10.944543), ('average_kl', 0.006207268816442826), ('average_policy_step_size', 1.0), ('explained_variance', 0.8071264717138438)]
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:396300 episode:785 R:152.2732688548364
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 50.79643), ('average_entropy', 10.944543), ('average_kl', 0.006207268816442826), ('average_policy_step_size', 1.0), ('explained_variance', 0.8071264717138438)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:144.00795878035314
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581

INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:404800 episode:802 R:142.30380089855583
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 56.204327), ('average_entropy', 10.9242115), ('average_kl', 0.006194862991105765), ('average_policy_step_size', 1.0), ('explained_variance', 0.7214885862943324)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:139.85047291773145
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:405300 episode:803 R:143.21716827210415
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 56.7736), ('average_entropy', 10.9242115), ('average_kl', 0.006194862991105765), ('average_policy_step_size', 1.0), ('explained_variance', 0.7214885862943324)]
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:405800 episode:804 R:127.19913625840488
INFO

INFO:pfrl.experiments.train_agent:statistics:[('average_value', 55.003326), ('average_entropy', 10.925545), ('average_kl', 0.006191581061745926), ('average_policy_step_size', 1.0), ('explained_variance', 0.9765327351001888)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:125.55316825972746
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:414300 episode:821 R:145.61209952275001
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 54.930866), ('average_entropy', 10.917374), ('average_kl', 0.006191581061745926), ('average_policy_step_size', 1.0), ('explained_variance', 0.9765327351001888)]
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:414800 episode:822 R:143.13789185710803
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 56.240807), ('average_entropy', 10.917374), ('average_kl', 0.006191581061745926), ('average_p

INFO:pfrl.agents.trpo:Line search iteration: 0 step size: 1.0
INFO:pfrl.agents.trpo:Surrogate objective improve: 0.04205893170001218
INFO:pfrl.agents.trpo:KL divergence: 0.005277795251458883
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:423300 episode:839 R:59.84289169641732
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 40.80189), ('average_entropy', 10.9101305), ('average_kl', 0.006178876750969461), ('average_policy_step_size', 1.0), ('explained_variance', 0.9307349164506944)]
INFO:pfrl.experiments.train_agent:evaluation episode 0 length:500 R:134.2821898578202
INFO:pfrl.experiments.train_agent:outdir:results\74b7ccfb1886c5e2ff20c581aa398dfd7298d1cf-00000000-1554415f step:423800 episode:840 R:82.55282263693898
INFO:pfrl.experiments.train_agent:statistics:[('average_value', 30.373318), ('average_entropy', 10.91127), ('average_kl', 0.006178876750969461), ('average_policy_step_size', 1.0), ('explained_va

In [ ]:
pfrl.experiments.train_agent_with_evaluation?